In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# Let's start by reading the data again and performing some basic preprocessing and cleaning.
data = pd.read_csv('/content/Data.csv')

# Convert InvoiceDate to datetime format for better handling
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Remove any negative or zero quantities
data = data[data['Quantity'] > 0]

# Remove any negative prices
data = data[data['Price'] > 0]

# Drop rows with missing values
data = data.dropna()

# Check the cleaned data
data.info(), data.describe()


<class 'pandas.core.frame.DataFrame'>
Index: 715163 entries, 0 to 944467
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      715163 non-null  object        
 1   StockCode    715163 non-null  object        
 2   Description  715163 non-null  object        
 3   Quantity     715163 non-null  int64         
 4   InvoiceDate  715163 non-null  datetime64[ns]
 5   Price        715163 non-null  float64       
 6   Customer ID  715163 non-null  float64       
 7   Country      715163 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 49.1+ MB


(None,
             Quantity                    InvoiceDate          Price  \
 count  715163.000000                         715163  715163.000000   
 mean       13.634735  2010-12-04 20:27:54.354740992       3.249971   
 min         1.000000            2009-12-01 07:45:00       0.001000   
 25%         2.000000            2010-06-16 12:48:00       1.250000   
 50%         6.000000            2010-11-16 14:01:00       1.950000   
 75%        12.000000            2011-06-07 13:44:00       3.750000   
 max     74215.000000            2011-11-09 12:49:00   10953.500000   
 std       117.982102                            NaN      30.717224   
 
          Customer ID  
 count  715163.000000  
 mean    15321.081514  
 min     12346.000000  
 25%     13969.000000  
 50%     15253.000000  
 75%     16791.000000  
 max     18287.000000  
 std      1693.938716  )

In [ ]:
# Additional preprocessing steps can include:

# Convert 'Customer ID' to int (as we've removed all NaN values, it's safe to convert to int)
data['Customer ID'] = data['Customer ID'].astype(int)

# Add additional useful columns for analysis
data['Year'] = data['InvoiceDate'].dt.year
data['Month'] = data['InvoiceDate'].dt.month
data['Day'] = data['InvoiceDate'].dt.day
data['Hour'] = data['InvoiceDate'].dt.hour
data['Revenue'] = data['Quantity'] * data['Price']  # Total revenue per transaction

# Remove outlier transactions to avoid skewing the data - we can use the IQR method for this
Q1 = data['Quantity'].quantile(0.25)
Q3 = data['Quantity'].quantile(0.75)
IQR = Q3 - Q1
# Filter out the outliers based on quantity
data = data[~((data['Quantity'] < (Q1 - 1.5 * IQR)) | (data['Quantity'] > (Q3 + 1.5 * IQR)))]

Q1_price = data['Price'].quantile(0.25)
Q3_price = data['Price'].quantile(0.75)
IQR_price = Q3_price - Q1_price
# Filter out the outliers based on price
data = data[~((data['Price'] < (Q1_price - 1.5 * IQR_price)) | (data['Price'] > (Q3_price + 1.5 * IQR_price)))]

# Let's also limit the visualization to the most recent full year of data to make trends more apparent
most_recent_year = data['Year'].max()
data = data[data['Year'] == most_recent_year]

# Now let's look at the cleaned data
data.info(), data.describe()


<class 'pandas.core.frame.DataFrame'>
Index: 255306 entries, 538375 to 944467
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      255306 non-null  object        
 1   StockCode    255306 non-null  object        
 2   Description  255306 non-null  object        
 3   Quantity     255306 non-null  int64         
 4   InvoiceDate  255306 non-null  datetime64[ns]
 5   Price        255306 non-null  float64       
 6   Customer ID  255306 non-null  int64         
 7   Country      255306 non-null  object        
 8   Year         255306 non-null  int32         
 9   Month        255306 non-null  int32         
 10  Day          255306 non-null  int32         
 11  Hour         255306 non-null  int32         
 12  Revenue      255306 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int32(4), int64(2), object(4)
memory usage: 23.4+ MB


(None,
             Quantity                    InvoiceDate          Price  \
 count  255306.000000                         255306  255306.000000   
 mean        7.782657  2011-07-01 02:49:03.596860160       2.195995   
 min         1.000000            2011-01-04 10:00:00       0.001000   
 25%         2.000000            2011-04-11 16:17:00       1.250000   
 50%         6.000000            2011-07-12 10:04:00       1.650000   
 75%        12.000000            2011-09-25 14:37:00       2.950000   
 max        27.000000            2011-11-09 12:49:00       7.500000   
 std         6.807423                            NaN       1.552470   
 
          Customer ID      Year          Month            Day           Hour  \
 count  255306.000000  255306.0  255306.000000  255306.000000  255306.000000   
 mean    15292.113284    2011.0       6.492158      15.365193      12.698417   
 min     12347.000000    2011.0       1.000000       1.000000       6.000000   
 25%     13949.000000    2011.0 

In [ ]:
import plotly.express as px

sampled_data = data.sample(n=500, random_state=1)

fig = px.parallel_coordinates(sampled_data,
                              dimensions=['Quantity', 'Price', 'Month', 'Day', 'Hour', 'Revenue'],
                              color='Revenue',
                              labels={'Quantity':'Quantity', 'Price':'Price',
                                      'Month':'Month', 'Day':'Day',
                                      'Hour':'Hour', 'Revenue':'Revenue'},
                              color_continuous_scale=px.colors.diverging.Tealrose,
                              color_continuous_midpoint=2)

fig.show()


In [ ]:
import plotly.express as px

sample_data_ts = data.sample(n=1000, random_state=1)

sales_data = sample_data_ts.groupby('InvoiceDate').agg({'Revenue': 'sum'}).reset_index()

fig = px.line(sales_data, x='InvoiceDate', y='Revenue', title='Time Series Decomposition of Sales',
              labels={'InvoiceDate': 'Date', 'Revenue': 'Revenue'},
              template='plotly_dark')

fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Revenue')

fig.show()


In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/content/Data.csv', on_bad_lines='skip')
except Exception as e:
    print("An error occurred:", e)

print(data.head(50))


In [ ]:
import pandas as pd
data = pd.read_csv('/content/Data.csv', on_bad_lines='skip')
categories = {
    'Christmas': 'CHRISTMAS',
    'Lighting': 'LIGHTS',
    'Frames': 'FRAME',
    'Ceramic': 'CERAMIC'
}

categorized_data = {}

for category, keyword in categories.items():
    is_in_category = data['Description'].str.contains(keyword, case=False, na=False)
    categorized_data[category] = data[is_in_category]


In [ ]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

category_labels = {
    'CHRISTMAS': 'Christmas Products',
    'LIGHTS': 'Lighting Products',
    'FRAME': 'Picture Frames',
    'CERAMIC': 'Ceramic Decorations'
}

data['Revenue'] = data['Quantity'] * data['Price']

data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

fig = make_subplots(specs=[[{"secondary_y": False}]])
for keyword, label in category_labels.items():
    category_data = data[data['Description'].str.contains(keyword, case=False, na=False)]
    category_revenue = category_data.groupby(category_data['InvoiceDate'].dt.date)['Revenue'].sum().reset_index()

    fig.add_trace(
        go.Scatter(
            x=category_revenue['InvoiceDate'],
            y=category_revenue['Revenue'],
            name=label,
            line=dict(width=2),
            mode='lines',
            showlegend=True
        )
    )

fig.update_layout(
    title_text='Time Series of Sales by Product Category',
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
    xaxis=dict(showline=True, showgrid=False, showticklabels=True, linecolor='white', linewidth=2),
    yaxis=dict(showline=True, showgrid=False, showticklabels=True, linecolor='white', linewidth=2),
    legend=dict(
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='white'
        ),
        bgcolor='black',
        bordercolor='Blue',
        borderwidth=2
    ),
)

for i, _ in enumerate(category_labels):
    fig.data[i].line.color = 'blue'

fig.update_layout(legend=dict(itemsizing='constant'))

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans

data_path = '/content/Data.csv'
data = pd.read_csv(data_path)

data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

data['Description'] = data['Description'].fillna('No Description')

data_clean = data.dropna(subset=['Customer ID'])

data_clean['Customer ID'] = data_clean['Customer ID'].astype(int)

clean_data_path = 'path_to_your_cleaned_file.csv'
data_clean.to_csv(clean_data_path, index=False)

import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans

data_path = '/content/path_to_your_cleaned_file.csv'
data = pd.read_csv(data_path)

customer_data = data.groupby('Customer ID').agg(
    Total_Sales=('Price', lambda x: (x * data.loc[x.index, 'Quantity']).sum()),
    Transaction_Count=('Invoice', 'nunique'),
    Average_Order_Value=('Price', 'mean')
)

customer_data_scaled = (customer_data - customer_data.mean()) / customer_data.std()
kmeans = KMeans(n_clusters=4, random_state=0)
customer_data_scaled['Segment'] = kmeans.fit_predict(customer_data_scaled)
customer_data['Segment'] = customer_data_scaled['Segment']
color_map = {0: 'High-Spending Customers', 1: 'Regular Customers', 2: 'low-spending customers', 3: 'very-jigh spending customers'}
customer_data['Color'] = customer_data['Segment'].map(color_map)
fig = px.scatter_3d(customer_data, x='Total_Sales', y='Transaction_Count', z='Average_Order_Value',
                    color='Color', title="Customer Segmentation", labels={
                        "Total_Sales": "Total Sales",
                        "Transaction_Count": "Transaction Count",
                        "Average_Order_Value": "Average Order Value"
                    })
fig.update_traces(marker=dict(size=5))
fig.show()

<ipython-input-5-b830dd391380>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [ ]:
category_keywords = {
    'Kitchenware': ['PIZZA PLATE', 'WALL CLOCK'],
    'Home Decor': ['FLYING DUCKS', 'WHITE FRAME', 'WOOD LETTERS', 'WREATH'],
    'Bath Accessories': ['BATH BUILDING', 'HOT WATER BOTTLE'],
    'Craft Supplies': ['CRAFT', 'TRINKET BOX', 'FELTCRAFT']
}

category_dataframes = {}

for category, keywords in category_keywords.items():
    category_mask = data['Description'].str.contains('|'.join(keywords), case=False, na=False)
    category_dataframes[category] = data[category_mask]

In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

all_categories_segmented = pd.DataFrame()

for category, df in category_dataframes.items():

    df['Total Sales'] = df['Quantity'] * df['Price']

    customer_data = df.groupby('Customer ID').agg(
        Total_Sales=('Total Sales', 'sum'),
        Transaction_Count=('Invoice', 'nunique'),
        Average_Order_Value=('Total Sales', 'mean')
    ).reset_index()

    scaler = StandardScaler()
    customer_data_scaled = scaler.fit_transform(customer_data[['Total_Sales', 'Transaction_Count', 'Average_Order_Value']])

    kmeans = KMeans(n_clusters=4, random_state=0)
    customer_data['Segment'] = kmeans.fit_predict(customer_data_scaled)

    customer_data['Category'] = category

    all_categories_segmented = pd.concat([all_categories_segmented, customer_data])

color_map = {0: 'High-Spending Customers', 1: 'Regular Customers', 2: 'Low-Spending Customers', 3: 'Very High-Spending Customers'}
all_categories_segmented['Segment_Label'] = all_categories_segmented['Segment'].map(color_map)

fig = px.scatter_3d(
    all_categories_segmented,
    x='Total_Sales',
    y='Transaction_Count',
    z='Average_Order_Value',
    color='Segment_Label',
    symbol='Category',
    hover_data=['Category'],
    title="Customer Segmentation within Product Categories",
    labels={
        "Total_Sales": "Total Sales",
        "Transaction_Count": "Transaction Count",
        "Average_Order_Value": "Average Order Value",
        "Segment_Label": "Customer Segment"
    }
)

fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
        xaxis_title='Total Sales',
        yaxis_title='Transaction Count',
        zaxis_title='Average Order Value'
    )
)

fig.show()


<ipython-input-4-976fcfdf0207>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

<ipython-input-4-976fcfdf0207>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change fr